In [97]:
import pandas as pd
import tldextract
import numpy as np
import json

In [63]:
iclab = pd.read_csv("cleaned_data/iclab_2018-09.csv")
iclab.head()

/Users/lilybhattacharjee/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,filename,server_t,country,as_number,schedule_name,url,dns,dns_reason,dns_all,dns_reason_all,http_status,block,body_len,http_reason,packet_updated,packet_reason,censored_updated
0,baseline-2018-09-01T000154.371069.json.bz2,2018-09-01T00:01:54.354Z,US,1249.0,country-sensitive-at,http://kinox.to/,NaN,NaN,NaN,NaN,403.0,False,3094.0,{},NaN,NaN,False
1,baseline-2018-09-01T000154.371069.json.bz2,2018-09-01T00:01:54.354Z,US,1249.0,country-sensitive-at,http://movie4k.to/,NaN,NaN,NaN,NaN,403.0,False,3098.0,{},NaN,NaN,False
2,baseline-2018-10-02T011527.678225.json.bz2,2018-09-01T00:02:11.575Z,KR,4766.0,country-sensitive-kr,4shared.com,false,NaN,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,baseline-2018-10-02T011527.678225.json.bz2,2018-09-01T00:02:11.575Z,KR,4766.0,country-sensitive-kr,news.bbc.co.uk,false,NaN,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,baseline-2018-10-02T011527.678225.json.bz2,2018-09-01T00:02:11.575Z,KR,4766.0,country-sensitive-kr,ngt.jinbo.net,false,NaN,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


For now, we only care about the 2-digit country code *country*, *url*, and *censored_updated*. Let's filter a cleaner table with just these columns.

In [64]:
iclab_clean = iclab[["country", "url", "censored_updated"]]
iclab_clean.head()

,country,url,censored_updated
0,US,http://kinox.to/,False
1,US,http://movie4k.to/,False
2,KR,4shared.com,False
3,KR,news.bbc.co.uk,False
4,KR,ngt.jinbo.net,False


We need to redo the combined_similarities.json and individual similarities files. First, let's find what countries are in the dataset.

In [5]:
iclab_clean.country.unique()

array(['US', 'KR', 'ES', 'ZA', 'CZ', 'PL', 'MY', 'RU', 'CN', 'TW', 'BG',
       'HK', 'RO', 'PE', 'HU', 'NO', 'MX', 'UA', 'NL', 'VN', 'JP', 'LT',
       'RS', 'AU', 'KE', 'SK', 'IN', 'CL', 'CA', 'LI', 'SG', 'ID', 'NZ',
       'LU', 'BZ', 'CO', 'TR', 'BR', 'SE', 'IS', 'FI', 'DZ', 'PT', 'DK',
       'IL', 'MD', 'AT', 'SC'], dtype=object)

In [57]:
cc_unique = iclab_clean.country.unique()

In [45]:
country_codes = pd.read_csv("https://raw.githubusercontent.com/daylight-lab/III/master/shared/data/country-codes/countries_codes_and_coordinates.csv").replace('"','', regex=True)
country_codes.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33,65
1,Albania,AL,ALB,8,41,20
2,Algeria,DZ,DZA,12,28,3
3,American Samoa,AS,ASM,16,-14.3333,-170
4,Andorra,AD,AND,20,42.5,1.6


In [46]:
country_codes.iloc[0]

Country                Afghanistan
Alpha-2 code                    AF
Alpha-3 code                   AFG
Numeric code                     4
Latitude (average)              33
Longitude (average)             65
Name: 0, dtype: object

In [47]:
country_codes.columns

Index(['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric code',
       'Latitude (average)', 'Longitude (average)'],
      dtype='object')

In [48]:
country_codes.dtypes

Country                object
Alpha-2 code           object
Alpha-3 code           object
Numeric code           object
Latitude (average)     object
Longitude (average)    object
dtype: object

In [50]:
country_codes['Alpha-2 code'] = country_codes['Alpha-2 code'].str.strip()

In [51]:
country_codes.loc[country_codes["Alpha-2 code"] == "AF"]

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33,65


In [55]:
all_countries = [country_codes.loc[country_codes["Alpha-2 code"] == u].iloc[0].Country for u in iclab_clean.country.unique()]
all_countries

['United States',
 'Korea, Republic of',
 'Spain',
 'South Africa',
 'Czech Republic',
 'Poland',
 'Malaysia',
 'Russian Federation',
 'China',
 'Taiwan, Province of China',
 'Bulgaria',
 'Hong Kong',
 'Romania',
 'Peru',
 'Hungary',
 'Norway',
 'Mexico',
 'Ukraine',
 'Netherlands',
 'Viet Nam',
 'Japan',
 'Lithuania',
 'Serbia',
 'Australia',
 'Kenya',
 'Slovakia',
 'India',
 'Chile',
 'Canada',
 'Liechtenstein',
 'Singapore',
 'Indonesia',
 'New Zealand',
 'Luxembourg',
 'Belize',
 'Colombia',
 'Turkey',
 'Brazil',
 'Sweden',
 'Iceland',
 'Finland',
 'Algeria',
 'Portugal',
 'Denmark',
 'Israel',
 'Moldova, Republic of',
 'Austria',
 'Seychelles']

In [65]:
iclab_clean["domain"] = [tldextract.extract(i).domain for i in iclab_clean["url"]]
iclab_clean["suffix"] = [tldextract.extract(i).suffix for i in iclab_clean["url"]]

/Users/lilybhattacharjee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/lilybhattacharjee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [66]:
iclab_clean.head()

,country,url,censored_updated,domain,suffix
0,US,http://kinox.to/,False,kinox,to
1,US,http://movie4k.to/,False,movie4k,to
2,KR,4shared.com,False,4shared,com
3,KR,news.bbc.co.uk,False,bbc,co.uk
4,KR,ngt.jinbo.net,False,jinbo,net


In [108]:
# recalculate combined_similarities.json
combined_similarities = {}
# for index, row in iclab_clean.iterrows():
#     country = row['country']
#     url = row['url']
#     censored = row['censored_updated']

def correct_country(c):
    if c == "United States":
        c = "United States of America"
    if c == 'Korea, Republic of':
        c = "South Korea"
    if c == 'Russian Federation':
        c = "Russia"
    if c == 'Taiwan, Province of China':
        c = "Taiwan"
    if c == 'Viet Nam':
        c = "Vietnam"
    if c == 'Moldova, Republic of':
        c = "Moldova"
    return c
    
    # format: "('Austria', 'BEL')": [{"similarity": 0.0496579382}]
for c1 in cc_unique:
    new_common_domains = pd.DataFrame(columns = ["Country 1", "Country 2", "Domain", "Suffix"])
    for c2 in cc_unique:
        if c1 != c2:
            country_1 = correct_country(country_codes.loc[country_codes["Alpha-2 code"] == c1].iloc[0].Country.strip())
            country_2 = correct_country(country_codes.loc[country_codes["Alpha-2 code"] == c2].iloc[0].Country.strip())
            
            c1_rows = iclab_clean.loc[iclab_clean["country"] == c1]
            c2_rows = iclab_clean.loc[iclab_clean["country"] == c2]
            
            joined_c1_c2 = pd.concat([c1_rows, c2_rows], axis = 0)
            joined_c1_c2["combined_site"] = joined_c1_c2["domain"] + "." + joined_c1_c2["suffix"]
            
            unique_sites = np.unique(joined_c1_c2["combined_site"])
            
            blocked_c1 = joined_c1_c2.loc[(joined_c1_c2["country"] == c1) & (joined_c1_c2["censored_updated"])]
            blocked_c2 = joined_c1_c2.loc[(joined_c1_c2["country"] == c2) & (joined_c1_c2["censored_updated"])]
            commonly_blocked = blocked_c1.merge(blocked_c2, how = 'inner', on = 'combined_site')
            blocked_both = commonly_blocked['combined_site'].unique()
            num_blocked_both = len(blocked_both)
            for b in blocked_both:
                suffix = b.split(".")[-1]
                domain = b[:(b.index(suffix) - 1)]
                new_common_domains = new_common_domains.append({"Country 1": country_1, "Country 2": country_2, "Domain": domain, "Suffix": suffix}, ignore_index = True)
            print(c1, c2, num_blocked_both)
            similarity = num_blocked_both / (len(unique_sites))
            
            country_2_alpha_2 = country_codes.loc[country_codes["Alpha-2 code"] == c2].iloc[0]["Alpha-3 code"].strip()
            if not country_2_alpha_2:
                country_2_alpha_2 = "MDA"
            k = "('" + country_1 + "', '" + country_2_alpha_2 + "')"
            combined_similarities[k] = [{"similarity": similarity}]
            print(k, similarity)
    new_common_domains.to_csv("new_common_domains/" + country_1 + "-common-domains.csv")

US KR 9
('United States of America', 'KOR') 0.003139169863969306
US ES 8
('United States of America', 'ESP') 0.003397027600849257
US ZA 9
('United States of America', 'ZAF') 0.003728251864125932
US CZ 7
('United States of America', 'CZE') 0.0030081650193382035
US PL 7
('United States of America', 'POL') 0.002892561983471074
US MY 9
('United States of America', 'MYS') 0.0033644859813084112
US RU 8
('United States of America', 'RUS') 0.0031360250882007056
US CN 8
('United States of America', 'CHN') 0.003228410008071025
US TW 8
('United States of America', 'TWN') 0.0030041306796845663
US BG 8
('United States of America', 'BGR') 0.0034379028792436614
US HK 8
('United States of America', 'HKG') 0.0032480714575720666
US RO 0
('United States of America', 'ROU') 0.0
US PE 8
('United States of America', 'PER') 0.003215434083601286
US HU 8
('United States of America', 'HUN') 0.003167062549485352
US NO 9
('United States of America', 'NOR') 0.003867640739149119
US MX 9
('United States of America',

('South Africa', 'ROU') 0.0
ZA PE 8
('South Africa', 'PER') 0.0032935364347468094
ZA HU 8
('South Africa', 'HUN') 0.0031885213232363493
ZA NO 9
('South Africa', 'NOR') 0.003909643788010426
ZA MX 9
('South Africa', 'MEX') 0.0036452004860267314
ZA UA 8
('South Africa', 'UKR') 0.0023228803716608595
ZA NL 9
('South Africa', 'NLD') 0.003842869342442357
ZA VN 10
('South Africa', 'VNM') 0.0038895371450797353
ZA JP 9
('South Africa', 'JPN') 0.00387263339070568
ZA LT 8
('South Africa', 'LTU') 0.0035087719298245615
ZA RS 7
('South Africa', 'SRB') 0.003027681660899654
ZA AU 8
('South Africa', 'AUS') 0.0034438226431338786
ZA KE 9
('South Africa', 'KEN') 0.0037067545304777594
ZA SK 8
('South Africa', 'SVK') 0.0034602076124567475
ZA IN 10
('South Africa', 'IND') 0.0038138825324180014
ZA CL 3
('South Africa', 'CHL') 0.0012234910277324632
ZA CA 4
('South Africa', 'CAN') 0.0017551557700745941
ZA LI 8
('South Africa', 'LIE') 0.0035087719298245615
ZA SG 7
('South Africa', 'SGP') 0.0029978586723768737
ZA 

MY BZ 5
('Malaysia', 'BLZ') 0.0019455252918287938
MY CO 8
('Malaysia', 'COL') 0.0030360531309297912
MY TR 8
('Malaysia', 'TUR') 0.0028021015761821367
MY BR 8
('Malaysia', 'BRA') 0.0017123287671232876
MY SE 8
('Malaysia', 'SWE') 0.003133568350959655
MY IS 8
('Malaysia', 'ISL') 0.003133568350959655
MY FI 8
('Malaysia', 'FIN') 0.0031237797735259665
MY DZ 0
('Malaysia', 'DZA') 0.0
MY PT 8
('Malaysia', 'PRT') 0.003133568350959655
MY DK 8
('Malaysia', 'DNK') 0.0031128404669260703
MY IL 9
('Malaysia', 'ISR') 0.0035252643948296123
MY MD 8
('Malaysia', 'MDA') 0.0027739251040221915
MY AT 0
('Malaysia', 'AUT') 0.0
MY SC 1
('Malaysia', 'SYC') 0.0003891050583657588
RU US 8
('Russia', 'USA') 0.0031360250882007056
RU KR 8
('Russia', 'KOR') 0.0026666666666666666
RU ES 8
('Russia', 'ESP') 0.0032626427406199023
RU ZA 8
('Russia', 'ZAF') 0.003194888178913738
RU CZ 7
('Russia', 'CZE') 0.0028618152085036794
RU PL 7
('Russia', 'POL') 0.0027821939586645467
RU MY 8
('Russia', 'MYS') 0.0028643036161833156
RU C

('Bulgaria', 'AUS') 0.0035778175313059034
BG KE 8
('Bulgaria', 'KEN') 0.003426124197002141
BG SK 8
('Bulgaria', 'SVK') 0.0036150022593764122
BG IN 9
('Bulgaria', 'IND') 0.0035101404056162248
BG CL 3
('Bulgaria', 'CHL') 0.0012674271229404308
BG CA 4
('Bulgaria', 'CAN') 0.001788109074653554
BG LI 8
('Bulgaria', 'LIE') 0.0036150022593764122
BG SG 7
('Bulgaria', 'SGP') 0.0030701754385964912
BG ID 6
('Bulgaria', 'IDN') 0.0024834437086092716
BG NZ 9
('Bulgaria', 'NZL') 0.00400355871886121
BG LU 8
('Bulgaria', 'LUX') 0.0036150022593764122
BG BZ 4
('Bulgaria', 'BLZ') 0.0017977528089887641
BG CO 8
('Bulgaria', 'COL') 0.003476749239461104
BG TR 8
('Bulgaria', 'TUR') 0.0031859816806053365
BG BR 8
('Bulgaria', 'BRA') 0.0018437427978796957
BG SE 8
('Bulgaria', 'SWE') 0.0035955056179775282
BG IS 8
('Bulgaria', 'ISL') 0.0035955056179775282
BG FI 8
('Bulgaria', 'FIN') 0.003590664272890485
BG DZ 0
('Bulgaria', 'DZA') 0.0
BG PT 8
('Bulgaria', 'PRT') 0.0035955056179775282
BG DK 8
('Bulgaria', 'DNK') 0.00

HU SK 8
('Hungary', 'SVK') 0.003280032800328003
HU IN 8
('Hungary', 'IND') 0.0028808066258552397
HU CL 3
('Hungary', 'CHL') 0.0011609907120743034
HU CA 4
('Hungary', 'CAN') 0.0016508460586050352
HU LI 8
('Hungary', 'LIE') 0.0033016921172100704
HU SG 7
('Hungary', 'SGP') 0.00282258064516129
HU ID 6
('Hungary', 'IDN') 0.0022935779816513763
HU NZ 9
('Hungary', 'NZL') 0.0036659877800407333
HU LU 8
('Hungary', 'LUX') 0.0032921810699588477
HU BZ 4
('Hungary', 'BLZ') 0.0016406890894175555
HU CO 8
('Hungary', 'COL') 0.0031961646024770275
HU TR 8
('Hungary', 'TUR') 0.002936857562408223
HU BR 8
('Hungary', 'BRA') 0.001762114537444934
HU SE 8
('Hungary', 'SWE') 0.0033030553261767133
HU IS 8
('Hungary', 'ISL') 0.0033030553261767133
HU FI 8
('Hungary', 'FIN') 0.0032921810699588477
HU DZ 0
('Hungary', 'DZA') 0.0
HU PT 8
('Hungary', 'PRT') 0.0033030553261767133
HU DK 8
('Hungary', 'DNK') 0.003281378178835111
HU IL 8
('Hungary', 'ISR') 0.0033030553261767133
HU MD 8
('Hungary', 'MDA') 0.002905920813657

...

LI HU 8
('Liechtenstein', 'HUN') 0.0033016921172100704
LI NO 8
('Liechtenstein', 'NOR') 0.0036117381489841984
LI MX 8
('Liechtenstein', 'MEX') 0.0033500837520938024
LI UA 8
('Liechtenstein', 'UKR') 0.002369668246445498
LI NL 8
('Liechtenstein', 'NLD') 0.0035382574082264487
LI VN 8
('Liechtenstein', 'VNM') 0.003236245954692557
LI JP 8
('Liechtenstein', 'JPN') 0.003576218149307108
LI LT 8
('Liechtenstein', 'LTU') 0.0036968576709796672
LI RS 7
('Liechtenstein', 'SRB') 0.0031631269769543608
LI AU 8
('Liechtenstein', 'AUS') 0.0035778175313059034
LI KE 8
('Liechtenstein', 'KEN') 0.003426124197002141
LI SK 8
('Liechtenstein', 'SVK') 0.0036150022593764122
LI IN 8
('Liechtenstein', 'IND') 0.003128666405944466
LI CL 3
('Liechtenstein', 'CHL') 0.0012674271229404308
LI CA 4
('Liechtenstein', 'CAN') 0.0018050541516245488
LI SG 7
('Liechtenstein', 'SGP') 0.003109729009329187
LI ID 6
('Liechtenstein', 'IDN') 0.0025348542458808617
LI NZ 8
('Liechtenstein', 'NZL') 0.0035587188612099642
LI LU 8
('Liecht

('New Zealand', 'TUR') 0.0031570639305445935
NZ BR 8
('New Zealand', 'BRA') 0.0018310826276035706
NZ SE 8
('New Zealand', 'SWE') 0.0035587188612099642
NZ IS 8
('New Zealand', 'ISL') 0.0035587188612099642
NZ FI 8
('New Zealand', 'FIN') 0.0035587188612099642
NZ DZ 0
('New Zealand', 'DZA') 0.0
NZ PT 8
('New Zealand', 'PRT') 0.0035587188612099642
NZ DK 8
('New Zealand', 'DNK') 0.0035587188612099642
NZ IL 8
('New Zealand', 'ISR') 0.0035587188612099642
NZ MD 8
('New Zealand', 'MDA') 0.0031019775106630476
NZ AT 0
('New Zealand', 'AUT') 0.0
NZ SC 1
('New Zealand', 'SYC') 0.00044483985765124553
LU US 8
('Luxembourg', 'USA') 0.0034379028792436614
LU KR 8
('Luxembourg', 'KOR') 0.002890173410404624
LU ES 8
('Luxembourg', 'ESP') 0.003573023671281822
LU ZA 8
('Luxembourg', 'ZAF') 0.0034812880765883376
LU CZ 7
('Luxembourg', 'CZE') 0.0031404217137729925
LU PL 7
('Luxembourg', 'POL') 0.0030381944444444445
LU MY 8
('Luxembourg', 'MYS') 0.0031237797735259665
LU RU 8
('Luxembourg', 'RUS') 0.0032666394446

TR NL 8
('Turkey', 'NLD') 0.003148366784730421
TR VN 8
('Turkey', 'VNM') 0.002888086642599278
TR JP 8
('Turkey', 'JPN') 0.0031708283789139914
TR LT 8
('Turkey', 'LTU') 0.0031859816806053365
TR RS 7
('Turkey', 'SRB') 0.0027877339705296694
TR AU 8
('Turkey', 'AUS') 0.0031720856463124504
TR KE 8
('Turkey', 'KEN') 0.0030429821224800305
TR SK 8
('Turkey', 'SVK') 0.0031859816806053365
TR IN 9
('Turkey', 'IND') 0.0031746031746031746
TR CL 3
('Turkey', 'CHL') 0.0011320754716981133
TR CA 4
('Turkey', 'CAN') 0.0015942606616181746
TR LI 8
('Turkey', 'LIE') 0.0031859816806053365
TR SG 7
('Turkey', 'SGP') 0.002730109204368175
TR ID 6
('Turkey', 'IDN') 0.002223869532987398
TR NZ 8
('Turkey', 'NZL') 0.0031570639305445935
TR LU 8
('Turkey', 'LUX') 0.0032038446135362435
TR BZ 4
('Turkey', 'BLZ') 0.0016019223067681217
TR CO 8
('Turkey', 'COL') 0.003109211037699184
TR BR 8
('Turkey', 'BRA') 0.0017297297297297297
TR SE 8
('Turkey', 'SWE') 0.0032038446135362435
TR IS 8
('Turkey', 'ISL') 0.00320384461353624

('Finland', 'MYS') 0.0031237797735259665
FI RU 8
('Finland', 'RUS') 0.0032520325203252032
FI CN 8
('Finland', 'CHN') 0.0033500837520938024
FI TW 8
('Finland', 'TWN') 0.0031262211801484957
FI BG 8
('Finland', 'BGR') 0.003590664272890485
FI HK 8
('Finland', 'HKG') 0.00337126000842815
FI RO 0
('Finland', 'ROU') 0.0
FI PE 8
('Finland', 'PER') 0.003357112882920688
FI HU 8
('Finland', 'HUN') 0.0032921810699588477
FI NO 8
('Finland', 'NOR') 0.0035778175313059034
FI MX 8
('Finland', 'MEX') 0.003369839932603201
FI UA 8
('Finland', 'UKR') 0.002354326074161271
FI NL 8
('Finland', 'NLD') 0.003547671840354767
FI VN 8
('Finland', 'VNM') 0.003216726980297547
FI JP 8
('Finland', 'JPN') 0.003576218149307108
FI LT 8
('Finland', 'LTU') 0.003590664272890485
FI RS 7
('Finland', 'SRB') 0.0031418312387791743
FI AU 8
('Finland', 'AUS') 0.0035778175313059034
FI KE 8
('Finland', 'KEN') 0.003426124197002141
FI SK 8
('Finland', 'SVK') 0.003590664272890485
FI IN 8
('Finland', 'IND') 0.0031237797735259665
FI CL 3
(

IL CN 8
('Israel', 'CHN') 0.0033500837520938024
IL TW 8
('Israel', 'TWN') 0.0031458906802988595
IL BG 8
('Israel', 'BGR') 0.0035778175313059034
IL HK 8
('Israel', 'HKG') 0.003401360544217687
IL RO 0
('Israel', 'ROU') 0.0
IL PE 8
('Israel', 'PER') 0.0033812341504649195
IL HU 8
('Israel', 'HUN') 0.0033030553261767133
IL NO 9
('Israel', 'NOR') 0.004063205417607223
IL MX 9
('Israel', 'MEX') 0.003778337531486146
IL UA 8
('Israel', 'UKR') 0.0023661638568470865
IL NL 9
('Israel', 'NLD') 0.003991130820399113
IL VN 8
('Israel', 'VNM') 0.0032064128256513026
IL JP 9
('Israel', 'JPN') 0.004023245417970496
IL LT 8
('Israel', 'LTU') 0.0036117381489841984
IL RS 7
('Israel', 'SRB') 0.0031305903398926656
IL AU 8
('Israel', 'AUS') 0.0035778175313059034
IL KE 9
('Israel', 'KEN') 0.0038412291933418692
IL SK 8
('Israel', 'SVK') 0.0035778175313059034
IL IN 8
('Israel', 'IND') 0.0031409501374165686
IL CL 3
('Israel', 'CHL') 0.0012626262626262627
IL CA 4
('Israel', 'CAN') 0.0018248175182481751
IL LI 8
('Israe

In [109]:
with open('../new-combined-similarities.json', 'w') as json_file:
    json.dump(combined_similarities, json_file)